In [4]:
import datetime
import pandas as pd
import numpy as np
from google.cloud import bigquery


import plotly
import plotly.express as px
from plotly import graph_objects as go


In [5]:
GROUP_NAME = "Non-Dairy"
# GROUP_NAME = 'Meat Substitutes'


# When using Streamlit, leave list empty and un-comment cells below
NORMALIZED_URL_LI = [
    'nutpods.com',
    'sodeliciousdairyfree.com',
    'coffeemate.com',
    'califiafarms.com',
    'silk.com',
]




BRAND_NAME_LI = [
    'Nutpods',
    'So Delicious',
    'Coffee Mate',
    'Califia',
    'Silk'
]

REV_COLS = ['source_name',
    'normalized_url',
    'review_source_id',
    'reviewer_source_id',
    'brand_name',
    'product_name',
    'product_source_id',
    'review_date',
    'review_rating',
    'review_content'
]

brand_name_di = dict(zip(NORMALIZED_URL_LI, BRAND_NAME_LI))



In [6]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

client = bigquery.Client(location="US", project="helio-staging")

Client creating using default project: helio-staging


In [7]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI),
        bigquery.ArrayQueryParameter("cols", "STRING", REV_COLS),
        bigquery.ArrayQueryParameter("brands", "STRING", BRAND_NAME_LI)
    ]
)

In [ ]:
job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI)
        ]
)
max_date_query = """
    SELECT max(execution_date)
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    where execution_date > DATE_ADD(current_date(), INTERVAL -30 DAY)
    and normalized_url IS NOT NULL
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

max_date = query_job.to_dataframe()['f0_'].astype('str')[0]
max_date

'2022-02-12'

In [9]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI),
        bigquery.ScalarQueryParameter("max_date", "STRING", max_date)
    ]
)
review_query = """
    SELECT
    canonical_brand_url,
    brand_source_id,
    retailer_name,
    source_type,
    
    FROM `helio-staging.offline_distribution.monthly_canonical_brand_store_distribution__1_0`
    WHERE canonical_brand_url IN UNNEST (@url)
    AND execution_date = @max_date
    """
query_job = client.query(
    review_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

reviews = query_job.to_dataframe()
reviews = reviews.loc[reviews.astype(str).drop_duplicates().index]


Forbidden: 403 Custom quota exceeded: Your usage exceeded the custom quota for QueryUsagePerDay, which is set by your administrator. For more information, see https://cloud.google.com/bigquery/cost-controls

(job ID: 2bef54ec-d749-4e1a-adc4-f4569c93eb81)

                                 -----Query Job SQL Follows-----                                  

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:    SELECT
   3:    canonical_brand_url,
   4:    brand_source_id,
   5:    retailer_name,
   6:    source_type,
   7:    
   8:    FROM `helio-staging.offline_distribution.monthly_canonical_brand_store_distribution__1_0`
   9:    WHERE canonical_brand_url IN UNNEST (@url)
  10:    AND execution_date = @max_date
  11:    
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |